
# <div style="text-align: center;">  Підготовка та аналіз даних  </div> #

# <div style="text-align: center;"> _Лабораторна робота №4, частина 1_ </div> #


## <div style="text-align: right;"> _ФБ-24 Бадрак Іван_ </div> ##





### 1. Розархівовуємо набір даних Individual household electric power consumption Data Set.
### Потім конвертуємо розархівований файл з .txt формату в .csv.
### Оброблюємо наш датафрейм, форматуємо дані та видаляємо пропущені значення.

In [15]:
import zipfile
import csv
import pandas as pd
from datetime import time as dt_time
import numpy as np
import time
import os
from IPython.display import display

file = open('household_power_consumption.csv', 'r')

def extract_zipfile():
    dir = "lab4"

    if os.path.exists('lab4/household_power_consumption.txt'):
        print ('Вже розархівовано')
        return
    else:
        with zipfile.ZipFile('individual+household+electric+power+consumption.zip', 'r') as zip_ref:
            zip_ref.extractall(dir)
        print("Успішно розархівовано.")
        return


# Pandas
def to_csv():
    txt_file = 'lab4/household_power_consumption.txt'
    csv_file = 'household_power_consumption.csv'

    with open(txt_file, 'r') as file_in, open(csv_file, 'w', newline='') as file_out:
        reader = csv.reader(file_in, delimiter=';')
        writer = csv.writer(file_out)

        for row in reader:
            writer.writerow(row)
    print(f"Успішно конвертовано {txt_file} у {csv_file}")

def pd_df():
    start_proc = time.time()
    columns = {'Global_active_power': float,
                'Global_reactive_power': float,
                'Voltage': float,
                'Global_intensity': float,
                'Sub_metering_1': float,
                'Sub_metering_2': float,
                'Sub_metering_3': float
    }

    na_values = ['?']
    df = pd.read_csv(file, dtype=columns, na_values=na_values)

    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
    df = df.drop(df.loc[(df['Sub_metering_1'] == 0) & (df['Sub_metering_2'] == 0) & (df['Sub_metering_3'] == 0)].index).dropna()
    final_time = time.time() - start_proc
    print ('Час обробки датафрейму за допомогою Pandas: ', final_time, 'c')
    return df

# Numpy

def file_np():
    txt_file = 'lab4/household_power_consumption.txt'
    data = np.genfromtxt(txt_file, delimiter=';')
    data[:,2:8] = data[:,2:8].astype(np.float64)
    return data

extract_zipfile()
to_csv()
df = pd_df()
np_arr = df.to_numpy()
print (df.isnull().sum())

print (file_np())


Вже розархівовано
Успішно конвертовано lab4/household_power_consumption.txt у household_power_consumption.csv
Час обробки датафрейму за допомогою Pandas:  7.255541563034058 c
Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64
[[   nan    nan    nan ...    nan    nan    nan]
 [   nan    nan  4.216 ...  0.     1.    17.   ]
 [   nan    nan  5.36  ...  0.     1.    16.   ]
 ...
 [   nan    nan  0.938 ...  0.     0.     0.   ]
 [   nan    nan  0.934 ...  0.     0.     0.   ]
 [   nan    nan  0.932 ...  0.     0.     0.   ]]


### Далі виконуємо завдання 1-5. Спочатку використовуємо dataframe, потім numpy. Також порівнюємо часові витрати на виконання процедур.
##### 1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.
##### 2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.
##### 3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.
##### 4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також
##### 5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [80]:
from datetime import datetime

# 1.
def task1df(df):
    proc_df = df[df['Global_active_power'] > 5.0]
    return proc_df

# 2.
def task2df(df):
    proc_df = df[df['Voltage'] > 235.0]
    return proc_df

# 3. 
def task3df(df):
    proc_df = df[(df['Global_intensity'] <= 20.0) & (df['Global_intensity'] >= 19.0)]
    proc_df = proc_df[(proc_df['Sub_metering_2'] > proc_df['Sub_metering_3']) & (proc_df['Sub_metering_2'] > proc_df['Sub_metering_1'])]
    return proc_df

# 4. 
def task4df(df):
    proc_df = df.sample(500000)
    # print (proc_df)
    # print (len(proc_df))
    avg_cons_values = proc_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
    return avg_cons_values

# 5. 
def task5df(df):
    proc_df = df[(df['Time'] > datetime.strptime('18:00:00', '%H:%M:%S').time()) & (df['Global_active_power'] > 6.0)]
    proc_df2 = proc_df[(proc_df['Sub_metering_2'] > proc_df['Sub_metering_1']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_3']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_2'].mean())]
    first_half = proc_df2.head(len(proc_df2) // 2)
    second_half = proc_df2.tail(len(proc_df2) // 2)
    aft_sampling = pd.concat([first_half.iloc[::3, :], second_half.iloc[::4, :]])
    return aft_sampling

# 1 numpy.
def task1numpy(np_arr):
    proc_df = np_arr[np_arr[:,2] > 5.0]
    return proc_df

# 2 numpy.
def task2numpy(np_arr):
    proc_df = np_arr[np_arr[:, 4] > 235]
    return proc_df

# 3 numpy.
def task3numpy(np_arr):
    proc_df = np_arr[(np_arr[:, 5] >= 19) & (np_arr[:, 5] <= 20)]
    proc_df = proc_df[(proc_df[:, 7] > proc_df[:, 8]) &
                    (proc_df[:, 7] > proc_df[:, 6])]
    return proc_df

# 4 numpy.
def task4numpy(np_arr, n=500000):
    sampled_indices = np_arr[np.random.choice(np_arr.shape[0], n, replace=True), 6:9]
    avg_cons_values = np.mean(sampled_indices, axis=0)
    return avg_cons_values

# 5 numpy.
def task5numpy(np_arr):
    filter_task = (np_arr[:, 1].astype('O').astype(dt_time) >= dt_time(18, 0, 0)) & \
    (np_arr[:, 2] > 6) & \
    (np_arr[:, 7] > np_arr[:, 6]) & \
    (np_arr[:, 7] > np_arr[:, 8])

    proc_df = np_arr[filter_task]
    first_half = proc_df[:len(proc_df) // 2]
    second_half = proc_df[len(proc_df) // 2:]
    aft_sampling = np.concatenate([first_half[::3, :], second_half[::4, :]])
    return aft_sampling


In [63]:
print("\nTask 1 DF")
start_time = time.time()
proc_df = task1df(df)
display(proc_df.head())
end_time = time.time() - start_time

print("\nTask 1 Numpy")
start_time_np = time.time()
proc_df = task1numpy(np_arr)
display(proc_df)
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time_np/end_time} разів\n ------------------------------------------ \n")


Task 1 DF


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
11,2006-12-16,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
12,2006-12-16,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0



Task 1 Numpy


array([[Timestamp('2006-12-16 00:00:00'), datetime.time(17, 25), 5.36,
        ..., 0.0, 1.0, 16.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 26), 5.374,
        ..., 0.0, 2.0, 17.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 27), 5.388,
        ..., 0.0, 1.0, 17.0],
       ...,
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 50), 5.172,
        ..., 0.0, 38.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 51), 5.75,
        ..., 0.0, 39.0, 17.0],
       [Timestamp('2010-11-25 00:00:00'), datetime.time(7, 21), 5.074,
        ..., 1.0, 2.0, 18.0]], dtype=object)

Час виконання:
DF: 0.017992496490478516 Numpy:0.030254364013671875 сек, DF швидше за Numpy у 1.6814989531709643 разів
 ------------------------------------------ 



In [96]:
print("\nTask 2 DF")
start_time = time.time()
proc_df = task2df(df)
display(proc_df.head())
end_time = time.time() - start_time

print("\nTask 2 Numpy")
start_time_np = time.time()
proc_df = task2numpy(np_arr)
display(proc_df[:5, :5])
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time_np/end_time} разів\n ------------------------------------------ \n")


Task 2 DF


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
5,2006-12-16,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
6,2006-12-16,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
7,2006-12-16,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
14,2006-12-16,17:38:00,4.054,0.422,235.24,17.6,0.0,1.0,17.0



Task 2 Numpy


array([[Timestamp('2006-12-16 00:00:00'), datetime.time(17, 28), 3.666,
        0.528, 235.68],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 29), 3.52,
        0.522, 235.02],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 30), 3.702,
        0.52, 235.09],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 31), 3.7,
        0.52, 235.22],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 38), 4.054,
        0.422, 235.24]], dtype=object)

Час виконання:
DF: 0.16334915161132812 Numpy:0.18287062644958496 сек, DF швидше за Numpy у 1.1195076597930922 разів
 ------------------------------------------ 



In [90]:
print("\nTask 3 DF")
start_time = time.time()
proc_df = task3df(df)
display(proc_df.head())
end_time = time.time() - start_time

print("\nTask 3 Numpy")
start_time_np = time.time()
proc_df = task3numpy(np_arr)
display(proc_df)
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time_np/end_time} разів\n ------------------------------------------ \n")


Task 3 DF


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
45,2006-12-16,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
460,2006-12-17,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
464,2006-12-17,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
475,2006-12-17,01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0
476,2006-12-17,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0



Task 3 Numpy


array([[Timestamp('2006-12-16 00:00:00'), datetime.time(18, 9), 4.464,
        ..., 0.0, 37.0, 16.0],
       [Timestamp('2006-12-17 00:00:00'), datetime.time(1, 4), 4.582,
        ..., 0.0, 13.0, 0.0],
       [Timestamp('2006-12-17 00:00:00'), datetime.time(1, 8), 4.618,
        ..., 0.0, 27.0, 0.0],
       ...,
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 55), 4.602,
        ..., 0.0, 40.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 56), 4.536,
        ..., 0.0, 39.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 57), 4.626,
        ..., 0.0, 39.0, 17.0]], dtype=object)

Час виконання:
DF: 0.01868605613708496 Numpy:0.06197929382324219 сек, DF швидше за Numpy у 3.3168740031897928 разів
 ------------------------------------------ 



In [66]:
print("\nTask 4 DF")
start_time = time.time()
proc_data = task4df(df)
display(proc_data)
end_time = time.time() - start_time

print("\nTask 4 Numpy")
start_time_np = time.time()
proc_data = task4numpy(np_arr)
display(proc_data)
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time_np/end_time} разів\n ------------------------------------------ \n")


Task 4 DF


Sub_metering_1    1.576890
Sub_metering_2    1.839378
Sub_metering_3    9.101500
dtype: float64


Task 4 Numpy


array([1.595334, 1.832088, 9.111006], dtype=object)

Час виконання:
DF: 0.19203758239746094 Numpy:0.3705432415008545 сек, DF швидше за Numpy у 1.9295350257739639 разів
 ------------------------------------------ 



In [81]:

print("\nTask 5 DF")
start_time = time.time()
proc_df = task5df(df)
display(proc_df)
end_time = time.time() - start_time

print("\nTask 5 Numpy")
start_time_np = time.time()
proc_df = task5numpy(np_arr)
display(proc_df)
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time_np/end_time} разів\n ------------------------------------------ \n")


Task 5 DF


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
41,2006-12-16,18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0
44,2006-12-16,18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0
17496,2006-12-28,21:00:00,6.376,0.252,235.94,27.4,1.0,53.0,17.0
17499,2006-12-28,21:03:00,7.906,0.286,235.93,33.6,1.0,73.0,17.0
17502,2006-12-28,21:06:00,7.184,0.000,235.70,30.4,2.0,73.0,17.0
...,...,...,...,...,...,...,...,...,...
2066467,2010-11-20,18:31:00,6.830,0.394,229.39,30.2,24.0,34.0,16.0
2066471,2010-11-20,18:35:00,6.784,0.364,228.38,30.0,21.0,35.0,16.0
2066475,2010-11-20,18:39:00,6.362,0.372,229.83,28.0,16.0,35.0,16.0
2066479,2010-11-20,18:43:00,6.324,0.376,229.93,27.8,14.0,35.0,17.0



Task 5 Numpy


array([[Timestamp('2006-12-16 00:00:00'), datetime.time(18, 5), 6.052,
        ..., 0.0, 37.0, 17.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(18, 8), 6.308,
        ..., 0.0, 36.0, 17.0],
       [Timestamp('2006-12-28 00:00:00'), datetime.time(20, 58), 6.386,
        ..., 1.0, 36.0, 17.0],
       ...,
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 38), 6.302,
        ..., 15.0, 34.0, 17.0],
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 42), 6.238,
        ..., 14.0, 35.0, 16.0],
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 46), 6.438,
        ..., 13.0, 39.0, 16.0]], dtype=object)

Час виконання:
DF: 0.06767940521240234 Numpy:0.15613412857055664 сек, DF швидше за Numpy у 2.306966618287373 разів
 ------------------------------------------ 



### Як показують порівняння, Pandas працює швидше й ефективніше в середньому в декілька разів за Numpy. Тому відносно питання часових витрат на виконання процедур перевага на боці _Pandas_